## Custom Job Manager

In this notebook, we show how you can modify the `openeo.extra.job_management.MultiBackendJobManager` to be able to:

- Postprocess your assets with Python libraries like `geopandas`, `xarray`, `rasterio`, ...
- Create a custom name / output path for your assets
- Write the STAC metadata to a STAC API

A basic example of the `MultiBackendJobManager` will be used. We will assume the user already has initialized a `JobDatabase` in Parquet format.

In [1]:
import openeo
import os
import pandas as pd
import pathlib
import pystac
import pystac_client
import requests
import shutil
import xarray as xr

from getpass import getpass
from requests.auth import AuthBase
from tempfile import NamedTemporaryFile
from typing import List

from openeo.extra.job_management import ParquetJobDatabase, MultiBackendJobManager

from openeo.rest.auth.oidc import (
    OidcClientInfo,
    OidcProviderInfo,
    OidcResourceOwnerPasswordAuthenticator,
)

Assume the user also has created a job database

In [2]:
df = pd.read_parquet('./job_df.parquet')

df 

,backend_name,start_date,end_date,spatial_extent
0,cdse,2021-10-10,2021-10-20,"{'east': 13.5, 'north': 52.6, 'south': 52.55, ..."
1,cdse,2021-01-20,2021-01-31,"{'east': 13.6, 'north': 52.7, 'south': 52.65, ..."


Initialize the `ParquetJobDatabase` to be passed on the `MultiBackendJobManager` later;

In [3]:
job_db = ParquetJobDatabase("./job_db.parquet")
job_db.initialize_from_df(df)

ParquetJobDatabase('job_db.parquet')

Define a very simple `start_job()` callable to be passed on to the `MultiBackendJobManager`. We'll just be extracting a few Sentinel-2 bands without any processing.

In [4]:
def start_job(row: pd.Series, connection_provider, connection: openeo.Connection, provider) -> openeo.BatchJob:
    
    s2_cube = connection.load_collection(
        collection_id="SENTINEL2_L2A",
        spatial_extent=row.spatial_extent,
        temporal_extent=[row.start_date , row.end_date],
        bands=["B02", "B03", "B04"],
        properties={"eo:cloud_cover": lambda x: x.lte(60)}
    )

    return s2_cube.create_job(
        title="Example Custom Job Manager",
        out_format="netcdf"
    )

Next, we will create a custom job manager class `CustomJobManager` which will be a subclass of `MultiBackendJobManager`, but which will overwrite the `on_job_done()` method. This is the method which is called once an openeo batch job has been successfully finished. We'll take over the code from the `MultiBackendJobManager`, but will add some additional things like:

- Being able to add postprocessing
- Adding custom name / output path for your assets
- Write away STAC metadata of the jobs to STAC API

In [5]:
class VitoStacApiAuthentication(AuthBase):
    """Class that handles authentication for the VITO STAC API. https://stac.openeo.vito.be/"""

    def __init__(self, **kwargs):
        self.username = kwargs.get("username")
        self.password = kwargs.get("password")

    def __call__(self, request):
        request.headers["Authorization"] = self.get_access_token()
        return request

    def get_access_token(self) -> str:
        """Get API bearer access token via password flow.

        Returns
        -------
        str
            A string containing the bearer access token.
        """
        provider_info = OidcProviderInfo(
            issuer="https://sso.terrascope.be/auth/realms/terrascope"
        )

        client_info = OidcClientInfo(
            client_id="terracatalogueclient",
            provider=provider_info,
        )

        if self.username and self.password:
            authenticator = OidcResourceOwnerPasswordAuthenticator(
                client_info=client_info, username=self.username, password=self.password
            )
        else:
            raise ValueError(
                "Credentials are required to obtain an access token. Please set STAC_API_USERNAME and STAC_API_PASSWORD environment variables."
            )

        tokens = authenticator.get_tokens()

        return f"Bearer {tokens.access_token}"



class STACApiInteraction:
    _ROOT_URL = "https://stac.openeo.vito.be"

    def __init__(self, collection_id: str, auth: AuthBase):
        self.collection_id = collection_id
        self.auth = auth

    def _prepare_item(self, item: pystac.Item):
        item.collection_id = self.collection_id
        if not item.get_links(pystac.RelType.COLLECTION):
            item.add_link(
                pystac.Link(rel=pystac.RelType.COLLECTION, target=item.collection_id)
            )
    
    def exists(self) -> bool:
        client = pystac_client.Client.open(self._ROOT_URL)
        return (
            len([c.id for c in client.get_collections() if c.id == self.collection_id])
            > 0
        )
    
    def _join_url(self, url_path: str) -> str:
        return str(self._ROOT_URL + "/" + url_path)
    
    def add_item(self, item: pystac.Item):
        if not self.exists():
            self.create_collection()

        self._prepare_item(item)

        url_path = f"collections/{self.collection_id}/items"
        response = requests.post(
            self._join_url(url_path), auth=self.auth, json=item.to_dict()
        )

        expected_status = [
            requests.status_codes.codes.ok,
            requests.status_codes.codes.created,
            requests.status_codes.codes.accepted,
        ]

        self._check_response_status(response, expected_status)

        return response
    
    def create_collection(self):
        spatial_extent = pystac.SpatialExtent([[-180, -90, 180, 90]])
        temporal_extent = pystac.TemporalExtent([[None, None]])
        extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)

        collection = pystac.Collection(
            id=self.collection_id,
            description=f"GFMap example for CustomJobManager",
            extent=extent,
        )

        collection.validate()
        coll_dict = collection.to_dict()

        default_auth = {
            "_auth": {
                "read": ["anonymous"],
                "write": ["stac-openeo-admin", "stac-openeo-editor"],
            }
        }

        coll_dict.update(default_auth)

        response = requests.post(
            self._join_url("collections"), auth=self.auth, json=coll_dict
        )

        expected_status = [
            requests.status_codes.codes.ok,
            requests.status_codes.codes.created,
            requests.status_codes.codes.accepted,
        ]

        self._check_response_status(response, expected_status)

        return response
    
    def _check_response_status(
        self, response: requests.Response, expected_status_codes: list[int]
    ):
        if response.status_code not in expected_status_codes:
            message = (
                f"Expecting HTTP status to be any of {expected_status_codes} "
                + f"but received {response.status_code} - {response.reason}, request method={response.request.method}\n"
                + f"response body:\n{response.text}"
            )

            raise Exception(message)

In [6]:
def generate_output_path(
    root_folder: pathlib.Path,
    row: pd.Series,
    asset_id: str,
) -> pathlib.Path:
    return root_folder / f"{row.id}_{asset_id}"

def post_job_action(job_items: List[pystac.Item], row: pd.Series, **kwargs) -> None:
    """Update the netcdf files with the start and end date of the job, and then write the items to STAC API.

    Parameters
    ----------
    job_items : List[pystac.Item]
        List of all STAC items that were created by the job
    row : pd.Series
        The row in the job database corresponding to the job
    """
    stac_api_interaction = STACApiInteraction(
        collection_id="gfmap_customjobmanager_example",
        auth=VitoStacApiAuthentication(
            username=getpass("STAC API username: "),
            password=getpass("STAC API password: "),
        ),
    )

    for idx, item in enumerate(job_items):
        item_asset_path = pathlib.Path(list(item.assets.values())[0].href)

        new_attributes = {
            "start_date": row.start_date,
            "end_date": row.end_date,
        }

        ds = xr.open_dataset(item_asset_path)

        ds = ds.assign_attrs(new_attributes)

        with NamedTemporaryFile(delete=False) as temp_file:
            ds.to_netcdf(temp_file.name)
            shutil.move(temp_file.name, item_asset_path)
    
        stac_api_interaction.add_item(item)
            

In [ ]:
class CustomJobManager(MultiBackendJobManager):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)  

    def on_job_done(self, job: openeo.BatchJob, row: pd.Series):
        """When a job is done, do the following:
        - Download the results to a directory, based on the generated_output_path callable
        - Postprocess the assets, based on the post_job_action callable
        - Write the STAC metadata to a STAC API

        Parameters
        ----------
        job : openeo.BatchJob
            The finished openeo batch job for which to handle the results
        row : pd.Series
            Row in the job database corresponding to the batch job that is done
        """
        job_products = {}
        job_results = job.get_results()
        asset_ids = [a.name for a in job_results.get_assets()]

        # Download and postprocess the assets
        for asset_id in asset_ids:
            asset = job_results.get_asset(asset_id)

            output_path = generate_output_path(self._root_dir, row, asset_id)
            output_path.parent.mkdir(parents=True, exist_ok=True)
            print(f'downloading asset {asset.name} to {output_path}')
            asset.download(output_path)
            print('Successfully downloaded asset')

            job_products[f"{job.job_id}_{asset_id}"] = [output_path]
        
        job_metadata = pystac.Collection.from_dict(job.get_results().get_metadata())
        job_items = []

        for item_metadata in job_metadata.get_all_items():
            item = pystac.read_file(item_metadata.get_self_href())
            asset_name = list(item.assets.values())[0].title
            asset_path = job_products[f"{job.job_id}_{asset_name}"][0]

            item.id = f'{row.id}_{asset_name}'

            if not len(item.assets.values()) == 1:
                raise ValueError("Each item should only contain one asset")  # This is not true in general, but it is for this use case. 

            for asset in item.assets.values():
                asset.href = str(
                    asset_path
                )  # Update the asset href to the output location set by the output_path_generator

            # Add the item to the the current job items.
            job_items.append(item)

        
        post_job_action(job_items, row)
        
        


Initiate and run the job manager

In [8]:
manager = CustomJobManager(root_dir="./results/")  
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()
manager.add_backend("cdse", connection=connection, parallel_jobs=2)

manager.run_jobs(start_job=start_job, 
                 job_db=job_db)

Authenticated using refresh token.
downloading asset openEO.nc to results/j-2503201101384455b27e11bde4f31165_openEO.nc
Successfully downloaded asset
downloading asset openEO.nc to results/j-250320110152474986cf5219df878217_openEO.nc
Successfully downloaded asset


defaultdict(int,
            {'job_db persist': 7,
             'track_statuses': 5,
             'job_db get_by_status': 1,
             'start_job call': 2,
             'job get status': 4,
             'job start': 2,
             'job launch': 2,
             'run_jobs loop': 5,
             'sleep': 5,
             'job describe': 7,
             'job started running': 1,
             'job finished': 2})

In [9]:
job_db = pd.read_parquet('./job_db.parquet')
job_db

,backend_name,start_date,end_date,spatial_extent,id,status,start_time,running_start_time,cpu,memory,duration,costs
0,cdse,2021-10-10,2021-10-20,"{'east': 13.5, 'north': 52.6, 'south': 52.55, ...",j-2503201101384455b27e11bde4f31165,finished,2025-03-20T11:01:38Z,2025-03-20T11:04:08Z,38.526273989 cpu-seconds,164888.625 mb-seconds,142 seconds,4
1,cdse,2021-01-20,2021-01-31,"{'east': 13.6, 'north': 52.7, 'south': 52.65, ...",j-250320110152474986cf5219df878217,finished,2025-03-20T11:01:53Z,None,119.758183234 cpu-seconds,813414.8020833333 mb-seconds,392 seconds,4


Remove the job database, so that this example can be repeated. DON'T DO THIS IN PRODUCTION :)

In [ ]:
os.remove("./job_db.parquet")

In [ ]:
resp = requests.delete('https://stac.openeo.vito.be/collections/gfmap_customjobmanager_example', auth=VitoStacApiAuthentication(username=getpass("STAC API username: "), password=getpass("STAC API password: ")))